In [1]:
import re
import csv, json
import pandas as pd

input_json = '/Users/verenakarina/Desktop/URECA/generate_dataset/200421_covidQA.json'

with open(input_json, "r", encoding = "utf-8") as json_file:
    objs = json.load(json_file)

In [5]:
import nltk
from nltk.tokenize import (sent_tokenize)

def para_split(para):
    final = []
    c = 0
    cc = 0
    p = ''
    sentences = sent_tokenize(para)
    for i in range(0, len(sentences)):
        s = sentences[i]
        words = s.split()
        c = c + len(words)
        if c <= 150:
            p = p + ' ' + s
            if i == len(sentences) - 1:
                final.append(p)
        else:
            cc = c - len(words)
            if abs(150 - c) < abs(150 - cc):
                p = p + ' ' + s
                final.append(p.strip())
                p = ''
                c = 0
            else:
                final.append(p.strip())
                p = s
                c = len(s.split())
    return final

#parse corpus
parsed_corpus = []
dict_titles = {}
for i in range(len(objs['data'])):
    unclean_context = objs['data'][i]['paragraphs'][0]['context']

    #generate title (match everything until line break)
    pattern = re.match(r'^.*', str(unclean_context))
    pattern = pattern.group()
    dict_titles[i] = pattern
    
    #clean spacing etc
    unclean_context = unclean_context.strip()
    unclean_context = unclean_context.replace('\n','')
    unclean_context = unclean_context.replace('\t','')
    unclean_context = re.sub(' +', ' ', unclean_context)
    
    #clean context
    keyword = "Abstract:"
    before_keyword, keyword, after_keyword = unclean_context.partition(keyword)
    
    #apply regex to remove intext citation
    unclean_corpus = re.sub(r'\[\d+\]','', after_keyword)

    #parse corpus into paragraphs
    parse_corpus = para_split(unclean_corpus)
    parsed_corpus.append(parse_corpus)
    

    
    
parsed_corpus

[['BACKGROUND: Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide. Given that the C-type lectin receptor, dendritic cell-specific ICAM-grabbing non-integrin-related (DC-SIGNR, also known as CD209L or liver/lymph node–specific ICAM-grabbing non-integrin (L-SIGN)), can interact with pathogens including HIV-1 and is expressed at the maternal-fetal interface, we hypothesized that it could influence MTCT of HIV-1. METHODS AND FINDINGS: To investigate the potential role of DC-SIGNR in MTCT of HIV-1, we carried out a genetic association study of DC-SIGNR in a well-characterized cohort of 197 HIV-infected mothers and their infants recruited in Harare, Zimbabwe. Infants harbouring two copies of DC-SIGNR H1 and/or H3 haplotypes (H1-H1, H1-H3, H3-H3) had a 3.6-fold increased risk of in utero (IU) (P = 0.013) HIV-1 infection and a 5.7-fold increased risk of intrapartum (IP) (P = 0.025) HIV-1 infection after adjusting for a number of maternal factors.',
 

In [3]:
def labels(corpus_index, answer):
    #return paragraph that contains the answer
    for paragraph in parsed_corpus[corpus_index]:
        if answer in paragraph:
            #clean unecessary keywords
            paragraph = paragraph.lower()
            paragraph = paragraph.strip()
            conditions = ['introduction:', 'introductions:' 'objective:', 'objectives:' 'conclusion:', 'conclusions:'
            , 'method:', 'methods:', 'background:', 'backgrounds:', 'result:', 'results:', 'result(s):', 'aim:']
            for i in conditions:
                paragraph = paragraph.replace(i, '')
            return paragraph


In [4]:
#construct triplet elements
label = []
q = []
a = []
label_len = []
title = []
for i in range(len(objs['data'])):
    total_qna = len(objs['data'][i]['paragraphs'][0]['qas'])
    for j in range(total_qna):
        #question answer
        question = objs['data'][i]['paragraphs'][0]['qas'][j]['question']
        answer = objs['data'][i]['paragraphs'][0]['qas'][j]['answers'][0]['text']
        quesiton = question.strip()
        answer = answer.strip()
        q.append(question.lower())
        a.append(answer.lower())
        
        #labels
        t = labels(i, answer)
        try:
            corpus_len = len(t.split())
        except:
            corpus_len = 0
        label.append(t)
        label_len.append(corpus_len)
        
        #title
        title.append(dict_titles[i].lower())
label

[' mother-to-child transmission (mtct) is the main cause of hiv-1 infection in children worldwide. given that the c-type lectin receptor, dendritic cell-specific icam-grabbing non-integrin-related (dc-signr, also known as cd209l or liver/lymph node–specific icam-grabbing non-integrin (l-sign)), can interact with pathogens including hiv-1 and is expressed at the maternal-fetal interface, we hypothesized that it could influence mtct of hiv-1. methods and findings: to investigate the potential role of dc-signr in mtct of hiv-1, we carried out a genetic association study of dc-signr in a well-characterized cohort of 197 hiv-infected mothers and their infants recruited in harare, zimbabwe. infants harbouring two copies of dc-signr h1 and/or h3 haplotypes (h1-h1, h1-h3, h3-h3) had a 3.6-fold increased risk of in utero (iu) (p = 0.013) hiv-1 infection and a 5.7-fold increased risk of intrapartum (ip) (p = 0.025) hiv-1 infection after adjusting for a number of maternal factors.',
 'the implica

In [202]:
df = pd.DataFrame(q, columns =['question'])
df['answer'] = a
df['label_len'] = label_len
df['positive_title'] = title
df['positive_label'] = label

In [203]:
#remove empty rows (answer not found in corpus)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df

,question,answer,label_len,positive_title,positive_label
0,what is the main cause of hiv-1 infection in c...,mother-to-child transmission (mtct) is the mai...,142,functional genetic variants in dc-signr are as...,mother-to-child transmission (mtct) is the ma...
1,what plays the crucial role in the mother to c...,dc-signr plays a crucial role in mtct of hiv-1...,153,functional genetic variants in dc-signr are as...,the implicated h1 and h3 haplotypes share two ...
2,how many children were infected by hiv-1 in 20...,"more than 400,000 children were infected world...",153,functional genetic variants in dc-signr are as...,the implicated h1 and h3 haplotypes share two ...
3,what is the role of c-c motif chemokine ligand...,"high copy numbers of ccl3l1, a potent hiv-1 su...",119,functional genetic variants in dc-signr are as...,genetic variants in ccr5 have been shown to in...
4,what is dc-genr and where is it expressed?,dendritic cell-specific icam-grabbing non-inte...,150,functional genetic variants in dc-signr are as...,"in the most heavilyaffected countries, such as..."
...,...,...,...,...,...
1116,"for the 2009 influenza pandemic, what were the...",that most cases of h1n1 influenza a virus infe...,156,it is unlikely that influenza viruses will cau...,"however, the truth was that most cases of h1n1..."
1117,what factors would contribute now to the fast...,"nowadays, we travel faster, and we travel more...",158,it is unlikely that influenza viruses will cau...,aside from one laboratory confirmed asymptotic...
1118,what factors would be responsible in future fo...,influenza virus infections are controllable an...,159,it is unlikely that influenza viruses will cau...,"however, we are now living in a technologicall..."
1119,what was the detected fatality rate of h7n9 av...,"the detected 32.14% (45/140, one case from tai...",158,it is unlikely that influenza viruses will cau...,aside from one laboratory confirmed asymptotic...


In [204]:
# generate dictionary of unique label
unique_label = {}
dict_value = []
index = 0
for i in df['positive_label']:
    #check if key-value pair already exist
    if i not in unique_label.values():
        unique_label[index] = i
        dict_value.append(i)
        index += 1
# unique_label
dict_value

[' mother-to-child transmission (mtct) is the main cause of hiv-1 infection in children worldwide. given that the c-type lectin receptor, dendritic cell-specific icam-grabbing non-integrin-related (dc-signr, also known as cd209l or liver/lymph node–specific icam-grabbing non-integrin (l-sign)), can interact with pathogens including hiv-1 and is expressed at the maternal-fetal interface, we hypothesized that it could influence mtct of hiv-1. methods and findings: to investigate the potential role of dc-signr in mtct of hiv-1, we carried out a genetic association study of dc-signr in a well-characterized cohort of 197 hiv-infected mothers and their infants recruited in harare, zimbabwe. infants harbouring two copies of dc-signr h1 and/or h3 haplotypes (h1-h1, h1-h3, h3-h3) had a 3.6-fold increased risk of in utero (iu) (p = 0.013) hiv-1 infection and a 5.7-fold increased risk of intrapartum (ip) (p = 0.025) hiv-1 infection after adjusting for a number of maternal factors.',
 'the implica

In [214]:
#generate negative context
import numpy as np
import random

neg_ctxs = []
k = 0
for id in df['positive_label']:
    dict_key = [i for i in range(len(unique_label))]

    #exclude positive ctxs
    exclude = int(dict_value.index(id))
    dict_key.remove(exclude)

    #generate negative ctxs
    index = random.Random(k).choice(dict_key)
    k+=1
    negative = unique_label[index]
    neg_ctxs.append(negative)
# neg_ctxs

In [215]:
# generate dictionary of title-label
dict_title = {}

for i in range(df.shape[0]):
    dict_title[df.positive_label[i]] = df.positive_title[i]

negative_title = []

for i in neg_ctxs:
    negative_title.append(dict_title.get(i))

negative_title

['chikungunya: a potentially emerging epidemic?',
 'inr-drug: predicting the interaction of drugs with nuclear receptors in cellular networking',
 'demographic variations of mers-cov infection among suspected and confirmed cases: an epidemiological analysis of laboratory-based data from riyadh regional laboratory',
 'sars to novel coronavirus – old lessons and new lessons',
 'sars to novel coronavirus – old lessons and new lessons',
 'viruses causing gastroenteritis: the known, the new and those beyond',
 'community-acquired pneumonia in children — a changing spectrum of disease',
 'potential rapid diagnostics, vaccine and therapeutics for 2019 novel coronavirus (2019-ncov): a systematic review',
 'mucosal immune responses induced by oral administration recombinant bacillus subtilis expressing the coe antigen of pedv in newborn piglets',
 'molecular and serological investigation of 2019-ncov infected patients: implication of multiple shedding routes',
 'changes in pulmonary tuberculosi

In [216]:
#reconstruct dataframe
df['negative_title'] = negative_title
df['negative_label'] = neg_ctxs

In [217]:
import os

path = '/Users/verenakarina/Desktop/URECA/generate_dataset/'
df.to_csv(os.path.join(path,'covid_dataset_revised.csv'),index=False)
# csv.to_csv('covid_dataset_revised.csv')